In [1]:
from transformers import Qwen2ForCausalLM, AutoTokenizer
import torch
from torch import nn
from copy import deepcopy
import numpy as np
import gc
import warnings
import sys
import os
from tqdm.auto import tqdm

src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from utils import *

from torch.utils.data import DataLoader,IterableDataset
from datasets import load_dataset
%load_ext autoreload
%autoreload 2

In [2]:
qwen_path = "D:\models\Qwen2.5-0.5B" # change it

qwen = Qwen2ForCausalLM.from_pretrained(qwen_path, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(qwen_path)

In [3]:
qwen = convert_to_Qwen2_ND(qwen).to("cuda")
qwen.eval();

In [4]:
# qwen = impacts_off(qwen) # comment this to test impacts

In [5]:
with torch.no_grad():
    output = qwen(tokenizer([
        "Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123, Hello World 123", 
        "Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456, Hello World 456",
    ], return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda"))

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
d:\Desktop\9сем\CreationOfIntelligentSystems_Multilingual_Interpretability\src\modules.py:183: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [6]:
qwen, _ = detect_domain_specific_neurons(
    qwen,
    tokenizer,
    dataloader=None,
    eps=1e-2,
    domain_name="eng", 
    reset_impacts=False,
    reset_dsn=True
)

In [7]:
class Trainer:
    def __init__(
        self, 
        model=None,
        train_config=None
    ):
        self.model = model
        self.train_config = train_config if train_config is not None else {}
    
    def train(
        self,
        train_loader=None,
        val_loader=None,
        model=None,
        only_domain_specific=True,
        domain=None,
        max_iters=None,
        optimizer=None,
        lr=None,
        ignore_index=None,
        evaluate_every=-1,  # -1 for no evaluation
    ):
        if model is not None:
            self.model = model
        if self.model is None:
            raise ValueError("No model provided")
        
        if lr is not None:
            self.train_config["lr"] = lr
        elif "lr" in self.train_config:
            lr = self.train_config["lr"]
        else:
            warnings.warn("No learning rate provided. Defaulting to 1e-3", UserWarning)
            lr = 1e-3
        
        if max_iters is not None:
            self.train_config["max_iters"] = max_iters
        elif "max_iters" in self.train_config:
            max_iters = self.train_config["max_iters"]
        else:
            max_iters = 1
            self.train_config["max_iters"] = max_iters
            warnings.warn("No max_iters provided. Defaulting to 100", UserWarning)

        if ignore_index is None:
            ignore_index = self.train_config.get("ignore_index", None)
        else:
            self.train_config["ignore_index"] = ignore_index
        
        loss_function = nn.CrossEntropyLoss(ignore_index=ignore_index, reduction="sum")
        
        if evaluate_every is not None:
            self.train_config["evaluate_every"] = evaluate_every
        elif "evaluate_every" in self.train_config:
            evaluate_every = self.train_config["evaluate_every"]
        else:
            warnings.warn("No evaluate_every provided. Defaulting to -1 (no evaluation)", UserWarning)
            evaluate_every = -1
        
        if domain is not None:
            self.train_config["domain"] = domain
        elif "domain" in self.train_config:
            domain = self.train_config["domain"]
        else:
            warnings.warn("No domain provided. Defaulting to 'eng'", UserWarning)
            domain = "eng"
        
        # IMPORTANT!
        if only_domain_specific:
            # calculate gradients only for layers with possible occurence of DSN
            dsn_model_grads_to_train(self.model)

        if optimizer is not None:
            if isinstance(optimizer, torch.optim.Optimizer):
                optimizer = optimizer([param for param in self.model.parameters() if param.requires_grad], lr=lr)
            else:
                self.train_config["optimizer"] = optimizer
        elif "optimizer" in self.train_config:
            opt_name = self.train_config["optimizer"]
        else:
            warnings.warn("No optimizer provided. Defaulting to Adam", UserWarning)
            opt_name = "Adam"

        if opt_name == "Adam":
            optimizer = torch.optim.Adam([param for param in self.model.parameters() if param.requires_grad], lr=lr)
        elif opt_name == "SGD":
            optimizer = torch.optim.SGD([param for param in self.model.parameters() if param.requires_grad], lr=lr)
        else:
            raise ValueError(f"Unknown optimizer: {opt_name}")
        
        losses = []
        metrics = []
        n_iter = 0
        device = next(self.model.parameters()).device

        self.model.train()

        with tqdm(range(max_iters), desc="Training iters") as pbar:
            for batch in train_loader:
                optimizer.zero_grad()
                
                output = self._forward(batch, device=device)["logits"]
                loss = loss_function(output.view(-1, output.shape[-1]), batch[0].to(device).view(-1))
                loss.backward()
                # IMPORTANT!
                if only_domain_specific:
                    dsn_model_mask_gradients(self.model, domain=domain)

                optimizer.step()

                losses.append(loss.detach().cpu().numpy())

                n_iter += len(batch)
                pbar.update(len(batch))
                
                if evaluate_every > 0 and n_iter >= (len(metrics) + 1) * evaluate_every:
                    eval_metrics = self.evaluate(val_loader)
                    metrics.append(eval_metrics)
                    self.model.train()
            
                if n_iter >= max_iters:
                    break

        self._metrics = metrics
        self._losses = losses

        return self.model
    
    def _forward(
        self, 
        batch, 
        device="cuda"
    ):
        input_ids, attention_mask = batch[0].to(device), batch[1].to(device) 
        preds = self.model(input_ids, attention_mask)

        return preds
    
    def _calulate_metrics(
        self, 
        output, 
        batch, 
        device="cuda"
    ):  
        metrics_to_calulate = self.train_config.get("eval_metrics", [])
        metrics = {}

        for m in metrics_to_calulate:
            if m == "loss" or m == "cross_entropy_loss":
                loss_function = nn.CrossEntropyLoss(ignore_index=self.train_config.get("ignore_index", None))
                metrics[m] = loss_function(output.view(-1, output.shape[-1]), batch[0].to(device).view(-1)).detach().cpu().numpy()
            else:
                warnings.warn(f"Metric: {m} NOT IMPLEMENTED", UserWarning)
            
        return metrics
    
    def evaluate(
        self,
        val_loader=None,
    ):
        if val_loader is not None:
            with torch.no_grad():
                self.model.eval()
                device = next(self.model.parameters()).device

                metrics = []
                # for batch in tqdm(val_loader, leave=False, desc="eval batch"):
                for batch in val_loader:
                    self.model.train()

                    output = self._forward(batch, device=device)["logits"]
                    
                    batch_metrics = self._calulate_metrics(output, batch, device)

                    metrics.append(batch_metrics)

                metrics_mean = {}
                for metric in metrics[0].keys():
                    metrics_mean[metric] = np.mean([m[metric] for m in metrics])
            
                return metrics_mean

        else:
            warnings.warn("No validation data provided, return empty val metrics", UserWarning)
            return {}    

In [8]:
collator = Collator(tokenizer)
dataset = load_dataset('yhavinga/ccmatrix', "en-ru", split='train', streaming=True, trust_remote_code=True)
data_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=False,
    collate_fn = collator,
    batch_sampler=None 
)

In [9]:
qwen.model.layers[0].self_attn.q_proj

Linear(in_features=896, out_features=896, bias=True)

In [10]:
trainer = Trainer(
    model=qwen, 
    train_config={
        "eval_metrics": ["loss"],
        "lr": 1e-4,
        "optimizer": "Adam",
        "max_iters": 500, 
        "ignore_index": tokenizer.pad_token_id
    }
)

qwen = trainer.train(
    train_loader=data_loader,
    val_loader=None,
    evaluate_every=-1,
    only_domain_specific=True,
    domain="eng"
)

Training iters:   0%|          | 0/500 [00:00<?, ?it/s]